## Run GPU baselines

In [ ]:
import os
import subprocess
from joblib import Parallel, delayed
from multiprocessing import Queue

from itertools import product

### Params

In [ ]:
# parameters
DEVICE_LIST = [0, 1]
N_JOBS = 4
N_TASKS_PER_DEVICE = 1 # since utilization is not high, some of the tasks could be ran on the same device
PYTHON = '../rapids-24.04/bin/python'

### Utils

In [ ]:
# fill GPU queue
QUEUE = Queue(maxsize=len(DEVICE_LIST) * N_TASKS_PER_DEVICE)
for i in range(len(DEVICE_LIST)):
    for _ in range(N_TASKS_PER_DEVICE):
        QUEUE.put(i)
    
# run script to execute the task
def get_script(path, runner, tuner, model, device):
    """
    Get run script for the task
    """
    command = f"""
    {PYTHON} run_experiment.py \
        --path {os.path.join('datasets', path)} \
        --njobs {N_JOBS} \
        --seed 42 \
        --device {device} \
        --runner {runner} \
        --tuner {tuner} \
        --model {model} \
        --config config.yaml
    """
    return command


def run(path, model, runner, tuner, ):
    """
    Run task
    """
    # get free GPU
    device = QUEUE.get()
    # generate script
    print('Task started')
    script = get_script(path, runner, tuner, model, device)
    print(script)
    # run task
    subprocess.check_output(script, shell=True, stderr=subprocess.STDOUT,)
    # back to queue
    QUEUE.put(device)
    
    return 

### Tasks list

In [ ]:
# tasks list
datasets = [
    
    'synth1', 
#     'hillstrom', 
#     'criteo',
#     'lenta',
#     'megafon',
]

# tuple: (type of model, run function, objective with param space)
models = [
    # py-boost baseline
    # ('pb_lc_f_t', 'pb', 'pb'), 
    # dragonnet
    ('dr', 'dr', 'dr'),
    # DESCN
    # ('dcn', 'dr', 'dcn')
]

# combine datasets and models
tasks = product(
    map(
        lambda x: x[0] + '_' + str(x[1]), product(datasets, range(5))
    ),
    models
)

### Run 

In [ ]:
with Parallel(len(DEVICE_LIST) * N_TASKS_PER_DEVICE, backend='threading') as p:
    p(delayed(run)(d, *m) for (d, m) in tasks)